# Rf 1 0 3_Interprfuncs
Basic functionality: interpreted functions and p.d.f.s



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:39 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"
#include "RooFitResult.h"
#include "RooGenericPdf.h"
#include "RooConstVar.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

----------------------------------------------------
Generic interpreted p.d.f.
 ====================================================

Declare observable x

In [3]:
RooRealVar x("x", "x", -20, 20);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Construct generic pdf from interpreted expression
 -------------------------------------------------------------------------------------------------

To construct a proper p.d.f, the formula expression is explicitly normalized internally by dividing
 it by a numeric integral of the expression over x in the range [-20,20]

In [4]:
RooRealVar alpha("alpha", "alpha", 5, 0.1, 10);
RooGenericPdf genpdf("genpdf", "genpdf", "(1+0.1*abs(x)+sin(sqrt(abs(x*alpha+0.1))))", RooArgSet(x, alpha));

Sample, fit and plot generic pdf
 ---------------------------------------------------------------

Generate a toy dataset from the interpreted p.d.f

In [5]:
RooDataSet *data = genpdf.generate(x, 10000);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(genpdf_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(genpdf_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)


Fit the interpreted p.d.f to the generated data

In [6]:
genpdf.fitTo(*data);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(genpdf_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 alpha        5.00000e+00  9.90000e-01    1.00000e-01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD         500           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=35708.9 FROM MIGRAD    STATUS=INITIATE        4 CALLS           5 TOTAL
               

Make a plot of the data and the p.d.f overlaid

In [7]:
RooPlot *xframe = x.frame(Title("Interpreted expression pdf"));
data->plotOn(xframe);
genpdf.plotOn(xframe);

[#1] INFO:NumericIntegration -- RooRealIntegral::init(genpdf_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)


-----------------------------------------------------------------------------------------------------------
Standard p.d.f adjust with interpreted helper function
 ==========================================================================================================
 Make a gauss(x,sqrt(mean2),sigma) from a standard RooGaussian

Construct standard pdf with formula replacing parameter
 ------------------------------------------------------------------------------------------------------------

Construct parameter mean2 and sigma

In [8]:
RooRealVar mean2("mean2", "mean^2", 10, 0, 200);
RooRealVar sigma("sigma", "sigma", 3, 0.1, 10);

Construct interpreted function mean = sqrt(mean^2)

In [9]:
RooFormulaVar mean("mean", "mean", "sqrt(mean2)", mean2);

Construct a gaussian g2(x,sqrt(mean2),sigma) ;

In [10]:
RooGaussian g2("g2", "h2", x, mean, sigma);

Generate toy data
 ---------------------------------

Construct a separate gaussian g1(x,10,3) to generate a toy gaussian dataset with mean 10 and width 3

In [11]:
RooGaussian g1("g1", "g1", x, RooConst(10), RooConst(3));
RooDataSet *data2 = g1.generate(x, 1000);

Fit and plot tailored standard pdf
 -------------------------------------------------------------------

Fit g2 to data from g1

In [12]:
RooFitResult *r = g2.fitTo(*data2, Save());
r->Print();

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mean2        1.00000e+01  5.00000e+00    0.00000e+00  2.00000e+02
     2 sigma        3.00000e+00  9.90000e-01    1.00000e-01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=5148.93 FROM MIGRAD    STATUS=INITIATE        8 CALLS           9 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX 

Plot data on frame and overlay projection of g2

In [13]:
RooPlot *xframe2 = x.frame(Title("Tailored Gaussian pdf"));
data2->plotOn(xframe2);
g2.plotOn(xframe2);

Draw all frames on a canvas

In [14]:
TCanvas *c = new TCanvas("rf103_interprfuncs", "rf103_interprfuncs", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
xframe->GetYaxis()->SetTitleOffset(1.4);
xframe->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
xframe2->GetYaxis()->SetTitleOffset(1.4);
xframe2->Draw();

Draw all canvases 

In [15]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()